In [22]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt

In [24]:
# Load all 317 files into a single DataFrame
# Read the data in chunks
datafolder = 'data/s3Files/'
data_files = os.listdir(datafolder)
data= pd.DataFrame()
for file in data_files:
    # print(file)
    data = pd.concat([data,pd.read_csv(datafolder+file,header=None,sep=',')])
data.drop(columns=[0,4],inplace=True)
data.dropna(inplace=True)
data.columns = ['current_1', 'current_2', 'current_3']

In [25]:
data.describe()

,current_1,current_2,current_3
count,3.170000e+06,3.170000e+06,3.170000e+06
mean,1.983090e+03,2.168260e+03,4.345496e+02
std,1.816999e+02,1.803412e+02,2.729833e+02
min,1.681000e+03,1.863000e+03,0.000000e+00
25%,1.787000e+03,1.977000e+03,1.440000e+02
50%,1.983000e+03,2.169000e+03,4.370000e+02
75%,2.180000e+03,2.360000e+03,7.250000e+02
max,2.292000e+03,2.497000e+03,9.070000e+02


In [26]:
data.reset_index(drop=True,inplace=True)

In [27]:
import copy
df = copy.deepcopy(data)

In [ ]:
# BrokenRotorBars = abs(current_1) - abs(current_2)
# BearingFault = abs(current_2) - abs(current_3)
# Eccentricity = abs(current_3) - abs(current_1)

# df['BrokenRotorBars'] = abs(df['current_1']) - abs(df['current_2'])
# df['BearingFault'] = abs(df['current_2']) - abs(df['current_3'])
# df['Eccentricity'] = abs(df['current_3']) - abs(df['current_1'])

In [ ]:
# df['rms_current'] = np.sqrt(np.square(df['current_1']) + np.square(df['current_2']) + np.square(df['current_3']))/3
# df['current_unbalance'] = np.abs((df['current_1'] - df['current_2']) / df['rms_current'])

In [28]:
# we might need to fft whole dataset since everything follow a signal pattern
fft_data = np.fft.fft(df, axis=0)

In [29]:
fft_data[0]

array([6.28639556e+09+0.j, 6.87338553e+09+0.j, 1.37752238e+09+0.j])

In [50]:
fundamental_freq = 50
harmonic_freqs = [2*fundamental_freq, 3*fundamental_freq, 4*fundamental_freq]

In [52]:
harmonic_freqs

[100, 150, 200]

In [54]:
[int(freq/fs * (len(fft_data)/2)) for freq in harmonic_freqs]

[15850, 23775, 31700]

Calculate harmonic magnitudes of the given dataset 

In [55]:
import numpy as np
import pandas as pd

# Set sampling frequency
fs = 10000


# Compute power spectral density
# This line computes the power spectral density (PSD) of the FFT by taking the absolute value of the FFT, squaring it, and dividing by the length of the current_data array. The PSD represents the distribution of power over the frequency range.
psd = np.abs(fft_data) ** 2 / len(df)


# Set frequency vector
# This line computes the frequency vector for the FFT by using the fftfreq() function from NumPy, which generates a vector of frequencies that correspond to each element of the FFT output.
freqs = np.fft.fftfreq(len(fft_data), 1 / fs) 

# Identify the fundamental frequency and its harmonics
fundamental_freq = 50 # or 60 Hz depending on the AC power supply frequency
harmonic_freqs = [2*fundamental_freq, 3*fundamental_freq, 4*fundamental_freq] # extract 2nd, 3rd and 4th harmonics
harmonic_indices = [int(freq/fs * (len(fft_data)/2)) for freq in harmonic_freqs] # compute indices of harmonic components
# harmonic_indices = [int(freq/fundamental_freq * (len(fft_1)//2)) for freq in harmonic_freqs if freq < fs/2] 

harmonic_magnitudes = np.abs(fft_data[harmonic_indices]) # extract magnitudes of harmonic components

# Print results
print('Fundamental frequency:', fundamental_freq, 'Hz')
print('Harmonic frequencies:', harmonic_freqs, 'Hz')
print('Harmonic magnitudes:', harmonic_magnitudes)


Fundamental frequency: 50 Hz
Harmonic frequencies: [100, 150, 200] Hz
Harmonic magnitudes: [[1099785.3481688   996971.17576751 1362701.88160155]
 [4788583.0103501  4700362.81987263 7178295.02667263]
 [ 243324.22633726  410422.70902971  722413.1563361 ]]


Calculating various other features 

In [ ]:
# Calculate THD
fundamental_index = int(fundamental_freq * (len(fft_data)/2) / fs)
thd = np.sqrt(np.sum(np.abs(fft_data[fundamental_index-2:fundamental_index+3])**2) - np.abs(fft_data[fundamental_index])**2) / np.abs(fft_data[fundamental_index])

# Calculate RMS current
rms_current = np.sqrt(np.mean(df**2))

# Calculate current unbalance
phase_angles = np.angle(fft_data[1:4, :], deg=True)
phase_angles_diff = np.diff(phase_angles, axis=0)
current_unbalance = np.max(np.abs(phase_angles_diff))

# Print results
print('Fundamental frequency:', fundamental_freq, 'Hz')
print('Harmonic frequencies:', harmonic_freqs, 'Hz')
print('Harmonic magnitudes:', harmonic_magnitudes)
print('THD:', thd)
print('RMS current:', rms_current)
print('Current unbalance:', current_unbalance)


creating anomaly


In [87]:
# Calculate the noise amplitude based on signal amplitude
noise_amplitude = 0.1*np.max(np.abs(df.loc[:999,:]))

# Add noise to the signal using amplitude scaling
noise = np.random.normal(0, noise_amplitude, df.loc[:999,:].shape)
noisy_signal = df.loc[:999,:] + noise

c:\Users\deepdesk\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [88]:
noisy_signal

,current_1,current_2,current_3
0,2278.670962,2344.192109,160.814288
1,1829.086990,2048.494432,45.381580
2,2448.411910,2300.903458,12.432574
3,1981.856134,2394.511509,115.398110
4,1842.779209,2434.134849,37.535929
...,...,...,...
995,2237.488102,1700.597476,645.298455
996,2064.302392,1854.696314,385.300779
997,2096.644228,1978.622265,582.626016
998,1829.329387,2063.196122,590.392824


In [92]:
anomalous_df = df.loc[:999,:]
anomalous_df.loc[:1000,:] = noise
anomalous_df.to_csv('anomalous_data.csv', index=False)


C:\Users\deepdesk\AppData\Local\Temp\ipykernel_8160\337689107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalous_df.loc[:1000,:] = noise


testing the anomaly

In [94]:
def get_features(df):

    fft_data = np.fft.fft(df, axis=0)
    # Set sampling frequency
    fs = 10000


    # Compute power spectral density
    # This line computes the power spectral density (PSD) of the FFT by taking the absolute value of the FFT, squaring it, and dividing by the length of the current_data array. The PSD represents the distribution of power over the frequency range.
    psd = np.abs(fft_data) ** 2 / len(df)


    # Set frequency vector
    # This line computes the frequency vector for the FFT by using the fftfreq() function from NumPy, which generates a vector of frequencies that correspond to each element of the FFT output.
    freqs = np.fft.fftfreq(len(fft_data), 1 / fs) 

    # Identify the fundamental frequency and its harmonics
    fundamental_freq = 50 # or 60 Hz depending on the AC power supply frequency
    harmonic_freqs = [2*fundamental_freq, 3*fundamental_freq, 4*fundamental_freq] # extract 2nd, 3rd and 4th harmonics
    harmonic_indices = [int(freq/fs * (len(fft_data)/2)) for freq in harmonic_freqs] # compute indices of harmonic components
    # harmonic_indices = [int(freq/fundamental_freq * (len(fft_1)//2)) for freq in harmonic_freqs if freq < fs/2] 

    harmonic_magnitudes = np.abs(fft_data[harmonic_indices]) # extract magnitudes of harmonic components

    # Print results
    # print('Fundamental frequency:', fundamental_freq, 'Hz')
    # print('Harmonic frequencies:', harmonic_freqs, 'Hz')
    # print('Harmonic magnitudes:', harmonic_magnitudes)
    # Calculate THD
    fundamental_index = int(fundamental_freq * (len(fft_data)/2) / fs)
    thd = np.sqrt(np.sum(np.abs(fft_data[fundamental_index-2:fundamental_index+3])**2) - np.abs(fft_data[fundamental_index])**2) / np.abs(fft_data[fundamental_index])

    # Calculate RMS current
    rms_current = np.sqrt(np.mean(df**2))

    # Calculate current unbalance
    phase_angles = np.angle(fft_data[1:4, :], deg=True)
    phase_angles_diff = np.diff(phase_angles, axis=0)
    current_unbalance = np.max(np.abs(phase_angles_diff))

    # Print results
    # print('Fundamental frequency:', fundamental_freq, 'Hz')
    # print('Harmonic frequencies:', harmonic_freqs, 'Hz')
    # print('Harmonic magnitudes:', harmonic_magnitudes)
    # print('THD:', thd)
    # print('RMS current:', rms_current)
    # print('Current unbalance:', current_unbalance)

    return {'fft_data' : fft_data,'harmonic_magnitudes' : harmonic_magnitudes ,'thd' : thd,'rms_current' : rms_current,'current_unbalance' : current_unbalance}


In [ ]:
new_fft,hm,nthd,rms,ubal = get_features(anomalous_df)


In [ ]:
np.concatenate((harmonic_magnitudes, current_unbalance, thd, rms_current), axis=1)

In [97]:
harmonic_magnitudes, hm

(array([[1099785.3481688 ,  996971.17576751, 1362701.88160155],
        [4788583.0103501 , 4700362.81987263, 7178295.02667263],
        [ 243324.22633726,  410422.70902971,  722413.1563361 ]]),
 array([[4683.32640125, 3782.7027863 , 3265.80834343],
        [4254.69676725, 4964.64955231, 1679.23047488],
        [8585.3574962 , 9424.45321275, 3116.62779596]]))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create a dataset with features and labels
X = np.concatenate((harmonic_magnitudes, current_unbalance, thd, rms_current), axis=1)
y = np.zeros(len(X))  # initialize all labels to 0 (normal)

# Set the threshold for anomaly detection
threshold = 0.2

# Determine the labels for anomalous data
for i in range(len(X)):
    for j in range(X.shape[1]):
        if abs(X[i][j] - benchmark_data[j]) > threshold:
            y[i] = 1  # set label to 1 (anomalous)

# Train a decision tree classifier
clf = DecisionTreeClassifier()
clf.fit(X, y)

# Use the classifier to predict labels for new data
# new_fft,hm,nthd,rms,ubal
X_new = np.concatenate((hm, ubal, nthd, rms), axis=1)
y_pred = clf.predict(X_new)

# Print the predicted labels
print("Predicted labels:", y_pred)
